In [4]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

## ======================================
## 1. Persiapan Data Sintetis
## ======================================

# Membuat dataset sintetis berdasarkan contoh yang diberikan
data = {
    'IPK': [3.8, 3.5, 2.8, 3.2, 2.5, 3.9, 3.0, 3.6, 2.9, 3.7, 3.1, 3.4, 2.7, 3.3, 2.4, 3.5, 3.0, 3.2, 2.6, 3.8],
    'Progress_Skripsi': [90, 75, 50, 80, 30, 95, 60, 85, 40, 92, 55, 78, 35, 70, 25, 88, 65, 72, 45, 96],
    'Kehadiran': [95, 85, 70, 90, 65, 98, 75, 88, 60, 94, 72, 82, 68, 80, 50, 90, 74, 84, 62, 97],
    'Jam_Organisasi': [5, 10, 15, 8, 20, 2, 12, 6, 18, 4, 14, 7, 16, 9, 20, 5, 11, 6, 17, 3],
    'Magang': [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
    'SKS': [144, 140, 130, 142, 125, 144, 135, 144, 128, 144, 132, 138, 126, 136, 120, 144, 134, 140, 124, 144],
    'Kategori': ['Lulus Cepat', 'Lulus Tepat Waktu', 'Butuh Perhatian', 'Lulus Tepat Waktu', 'Butuh Perhatian',
                'Lulus Cepat', 'Butuh Perhatian', 'Lulus Tepat Waktu', 'Butuh Perhatian', 'Lulus Cepat',
                'Butuh Perhatian', 'Lulus Tepat Waktu', 'Butuh Perhatian', 'Lulus Tepat Waktu', 'Butuh Perhatian',
                'Lulus Cepat', 'Butuh Perhatian', 'Lulus Tepat Waktu', 'Butuh Perhatian', 'Lulus Cepat']
}

df = pd.DataFrame(data)

# Konversi kategori ke nilai numerik
kategori_map = {'Lulus Cepat': 0, 'Lulus Tepat Waktu': 1, 'Butuh Perhatian': 2}
df['Kategori_Encoded'] = df['Kategori'].map(kategori_map)

## ======================================
## 2. Fuzzy Inference System (FIS)
## ======================================

# Variabel input fuzzy
ipk = ctrl.Antecedent(np.arange(0, 4.1, 0.1), 'ipk')
progress = ctrl.Antecedent(np.arange(0, 101, 1), 'progress')
kehadiran = ctrl.Antecedent(np.arange(0, 101, 1), 'kehadiran')
organisasi = ctrl.Antecedent(np.arange(0, 21, 1), 'organisasi')
sks = ctrl.Antecedent(np.arange(100, 151, 1), 'sks')

# Variabel output fuzzy
kelulusan = ctrl.Consequent(np.arange(0, 3, 1), 'kelulusan')

# Fungsi keanggotaan untuk variabel input
ipk['rendah'] = fuzz.trapmf(ipk.universe, [0, 0, 2.5, 2.8])
ipk['sedang'] = fuzz.trimf(ipk.universe, [2.5, 3.0, 3.5])
ipk['tinggi'] = fuzz.trapmf(ipk.universe, [3.0, 3.5, 4.0, 4.0])

progress['rendah'] = fuzz.trapmf(progress.universe, [0, 0, 40, 60])
progress['sedang'] = fuzz.trimf(progress.universe, [40, 70, 90])
progress['tinggi'] = fuzz.trapmf(progress.universe, [70, 90, 100, 100])

kehadiran['rendah'] = fuzz.trapmf(kehadiran.universe, [0, 0, 65, 75])
kehadiran['sedang'] = fuzz.trimf(kehadiran.universe, [65, 80, 90])
kehadiran['tinggi'] = fuzz.trapmf(kehadiran.universe, [80, 90, 100, 100])

organisasi['sedikit'] = fuzz.trapmf(organisasi.universe, [0, 0, 5, 10])
organisasi['sedang'] = fuzz.trimf(organisasi.universe, [5, 10, 15])
organisasi['banyak'] = fuzz.trapmf(organisasi.universe, [10, 15, 20, 20])

sks['kurang'] = fuzz.trapmf(sks.universe, [100, 100, 130, 135])
sks['cukup'] = fuzz.trimf(sks.universe, [130, 140, 150])
sks['lengkap'] = fuzz.trapmf(sks.universe, [140, 144, 150, 150])

# Fungsi keanggotaan untuk variabel output
kelulusan['cepat'] = fuzz.trimf(kelulusan.universe, [0, 0, 1])
kelulusan['tepat'] = fuzz.trimf(kelulusan.universe, [0, 1, 2])
kelulusan['terlambat'] = fuzz.trimf(kelulusan.universe, [1, 2, 2])

# Aturan fuzzy
rule1 = ctrl.Rule(ipk['tinggi'] & progress['tinggi'] & kehadiran['tinggi'] & organisasi['sedikit'] & sks['lengkap'], kelulusan['cepat'])
rule2 = ctrl.Rule(ipk['tinggi'] & progress['tinggi'] & kehadiran['tinggi'] & organisasi['sedang'] & sks['lengkap'], kelulusan['cepat'])
rule3 = ctrl.Rule(ipk['sedang'] & progress['sedang'] & kehadiran['sedang'] & organisasi['sedang'] & sks['cukup'], kelulusan['tepat'])
rule4 = ctrl.Rule(ipk['sedang'] & progress['sedang'] & kehadiran['tinggi'] & organisasi['sedikit'] & sks['cukup'], kelulusan['tepat'])
rule5 = ctrl.Rule(ipk['rendah'] | progress['rendah'] | kehadiran['rendah'], kelulusan['terlambat'])
rule6 = ctrl.Rule(organisasi['banyak'] & (ipk['rendah'] | progress['rendah']), kelulusan['terlambat'])
rule7 = ctrl.Rule(sks['kurang'] & (ipk['rendah'] | progress['rendah']), kelulusan['terlambat'])

# Sistem kontrol fuzzy
kelulusan_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
kelulusan_sim = ctrl.ControlSystemSimulation(kelulusan_ctrl)

# Fungsi untuk mendapatkan output fuzzy
def fuzzy_predict(row):
    try:
        # Pastikan input dalam range yang valid
        inputs = {
            'ipk': max(0, min(4.0, row['IPK'])),
            'progress': max(0, min(100, row['Progress_Skripsi'])),
            'kehadiran': max(0, min(100, row['Kehadiran'])),
            'organisasi': max(0, min(20, row['Jam_Organisasi'])),
            'sks': max(100, min(150, row['SKS']))
        }
        
        for key, val in inputs.items():
            kelulusan_sim.input[key] = val
            
        kelulusan_sim.compute()
        
        # Cek jika output berhasil dihasilkan
        if not kelulusan_sim.output:
            print(f"Warning: Tidak ada output fuzzy untuk row {row.name}")
            return 1.5  # Nilai tengah sebagai default
            
        return kelulusan_sim.output['kelulusan']
        
    except Exception as e:
        print(f"Error processing row {row.name}: {str(e)}")
        return 1.5  # Nilai default jika terjadi error

# Tambahkan hasil fuzzy ke dataframe
df['Fuzzy_Output'] = df.apply(fuzzy_predict, axis=1)

## ======================================
## 3. Neural Network (NN)
## ======================================

# Gabungkan output fuzzy dengan fitur asli
X = df[['IPK', 'Progress_Skripsi', 'Kehadiran', 'Jam_Organisasi', 'Magang', 'SKS', 'Fuzzy_Output']]
y = df['Kategori_Encoded']

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Bangun model neural network
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Latih model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=8, validation_split=0.1)

# Evaluasi model
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f'\nAkurasi Test: {test_acc:.2f}')

# Prediksi
y_pred = np.argmax(model.predict(X_test_scaled), axis=1)
print(classification_report(y_test, y_pred, target_names=kategori_map.keys()))

## ======================================
## 4. Fungsi Prediksi + Tips
## ======================================

def predict_with_tips(input_data):
    # Konversi ke dataframe
    input_df = pd.DataFrame([input_data], columns=X.columns[:-1])
    
    # Hitung output fuzzy
    fuzzy_out = fuzzy_predict(input_df.iloc[0])
    input_df['Fuzzy_Output'] = fuzzy_out
    
    # Normalisasi input
    input_scaled = scaler.transform(input_df)
    
    # Prediksi dengan NN
    pred_proba = model.predict(input_scaled)[0]
    pred_class = np.argmax(pred_proba)
    
    # Konversi ke kategori
    inv_kategori_map = {v: k for k, v in kategori_map.items()}
    kategori_pred = inv_kategori_map[pred_class]
    
    # Generate tips
    tips = []
    if input_data['IPK'] < 2.8:
        tips.append("Perbaiki IPK dengan belajar lebih intensif!")
    if input_data['Progress_Skripsi'] < 60:
        tips.append("Tingkatkan progress skripsi!")
    if input_data['Kehadiran'] < 75:
        tips.append("Perbanyak kehadiran kuliah!")
    if input_data['Jam_Organisasi'] > 12:
        tips.append("Kurangi jam organisasi, fokus skripsi!")
    if input_data['SKS'] < 135:
        tips.append("Selesaikan SKS yang kurang!")
    if input_data['Magang'] == 0:
        tips.append("Pertimbangkan magang untuk pengalaman tambahan!")
    
    if not tips:
        tips.append("Anda berada di jalur yang tepat! Pertahankan.")
    
    return {
        'Kategori': kategori_pred,
        'Tips': tips,
        'Probabilitas': {
            'Lulus Cepat': float(pred_proba[0]),
            'Lulus Tepat Waktu': float(pred_proba[1]),
            'Butuh Perhatian': float(pred_proba[2])
        }
    }

# Contoh penggunaan
sample_input = {
    'IPK': 3.2,
    'Progress_Skripsi': 78,
    'Kehadiran': 82,
    'Jam_Organisasi': 7,
    'Magang': 1,
    'SKS': 138
}

result = predict_with_tips(sample_input)
print("\nHasil Prediksi:")
print(f"Kategori: {result['Kategori']}")
print("Probabilitas:")
for k, v in result['Probabilitas'].items():
    print(f"- {k}: {v:.2f}")
print("Tips:")
for tip in result['Tips']:
    print(f"- {tip}")

Epoch 1/50


C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 225ms/step - accuracy: 0.3631 - loss: 1.1407 - val_accuracy: 0.0000e+00 - val_loss: 0.9763
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.4107 - loss: 1.0071 - val_accuracy: 0.0000e+00 - val_loss: 0.9252
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5893 - loss: 0.9778 - val_accuracy: 0.5000 - val_loss: 0.8781
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.8155 - loss: 0.9002 - val_accuracy: 0.5000 - val_loss: 0.8344
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7321 - loss: 0.9048 - val_accuracy: 0.5000 - val_loss: 0.7841
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.7262 - loss: 0.8495 - val_accuracy: 1.0000 - val_loss: 0.7360
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8155 - loss: 0.7870 - val_accuracy: 1.0000 - val_loss: 0.6929
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.8155 - loss: 0.7634 - val_accuracy: 1.0000 - val_loss: 0.6547
Ep

ValueError: Number of classes, 2, does not match size of target_names, 3. Try specifying the labels parameter